# <center>Applied Data Science Capstone - Peer-graded Assignment 2</center>
# <center>Segmenting and Clustering Neighborhoods in Toronto </center>

### 1. Get postal codes from Wikipedia

In [1]:
import requests

# Get html content:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content

In [2]:
from scrapy import Selector

# Parse html and find tags containing requested info:
sel = Selector(text=html)
rows = sel.css('table.wikitable > tbody > tr')

In [3]:
# Create list to store parsed rows:
table_rows = []

# Iterate over rows and extract information:
for row in rows[1:]:  # Ommit the first row, it is the table header
    text = row.css('td ::text').extract()  # returns list of strings, could be more than 3 items
    postcode = text[0]
    borough  = text[1]
    neighbor = text[2].replace('\n','')
    
    # If Neighbourhood isn't assigned, replace it with Borough name:
    neighbor = borough if neighbor == 'Not assigned' else neighbor
    
    # Append only codes where Borough is assigned:
    if borough != 'Not assigned':
        table_rows.append(
            {'Postcode'     : postcode,
             'Borough'      : borough,
             'Neighbourhood': neighbor
            }
        )

In [4]:
import pandas as pd

# Convert the list into the pandas DataFrame:
df = pd.DataFrame(table_rows, columns=['Postcode','Borough','Neighbourhood'])
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
# Group neighbourhoods by Postcodes:
result = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).to_frame().reset_index()
result.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
